https://www.kaggle.com/datasets/andrewmvd/data-analyst-jobs

In [1]:
import pandas as pd
import numpy as np

In [20]:
df = pd.read_csv('DataAnalyst.csv')
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,-1
3,3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,-1
4,4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,True


In this dataset, null values are indicated by a "-1". Since this won't show up as a null value during data exploration, all values equalling -1 will be converted to NaNs. 

In [28]:
df = df.replace('-1', None)
df = df.replace(-1, None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2253 entries, 0 to 2252
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         2253 non-null   int64 
 1   Job Title          2253 non-null   object
 2   Salary Estimate    2252 non-null   object
 3   Job Description    2253 non-null   object
 4   Rating             1981 non-null   object
 5   Company Name       2252 non-null   object
 6   Location           2253 non-null   object
 7   Headquarters       2081 non-null   object
 8   Size               2090 non-null   object
 9   Founded            1593 non-null   object
 10  Type of ownership  2090 non-null   object
 11  Industry           1900 non-null   object
 12  Sector             1900 non-null   object
 13  Revenue            2090 non-null   object
 14  Competitors        521 non-null    object
 15  Easy Apply         80 non-null     object
dtypes: int64(1), object(15)
memory usage: 281.

The attributes "Competitors" and "Easy Apply" will be dropped, as they contain mostly null values and are of little interest to this assessment. 

Moving forward, this analysis will make the presumption that job seekers generally care about some of these remaining attributes far more than others. The goal in this feature selection will be to best map this preference in a workable way. The additional attributes that will be dropped are:
- "Company Name": important to the job seeker, but not useful in clustering when there are no associated company features by which to compare companies to each other. 
- "Headquarters": Most job seekers likely do not hold strong preferences here.
- "Type of ownership": Heavily imbalanced and usually of relatively little consideration.
- "Industry": Could be rather useful, but highly obfuscated by the fact that many jobs are listed through a staffing agency that are irrelevant to the job itself, and that generic categories like "IT Serivces" are often used. Additionally, the "Sector" attribute already captures much of the desired information.
- "Revenue": A plurality of these are unknown, and startup status will be captured by other features. 

- "Unnamed: 0": Will be renamed as "JobID" for the purpose of this analysis. 
- "Job Title": Will need to be reduced into more easily useable bins, as many job titles are tautamount to each other and differ primarily in a semantic sense. 
- "Salary Estimate": Will be converted into a more useable numeric format consisting of the median between the two values.
- "Job Description": Will be vectorized as to be useable by data analysis tools.
- "Founded": Will be converted to "CompanyAge", then binned. This should be useful as job seekers likely have a preference in working for a recent startup or not. 

In [33]:
# Sample of attribute exploration. This has been performed for all features, but only a sample has been kept
# as to reduce the amount of space taken up by this analysis. 
df['Industry'].value_counts()

IT Services                            325
Staffing & Outsourcing                 323
Health Care Services & Hospitals       151
Computer Hardware & Software           111
Consulting                             111
                                      ... 
Truck Rental & Leasing                   1
News Outlet                              1
Catering & Food Service Contractors      1
Chemical Manufacturing                   1
Hotels, Motels, & Resorts                1
Name: Industry, Length: 88, dtype: int64